In [1]:
import pandas as pd 

df = pd.read_pickle("data/matches.pkl")
df

,map_number,match_id,dire_team,dire_heroes,dire_win,radiant_team,radiant_heroes,radiant_win,winner
0,Map #1,6130055039,Team Aster,"[Luna, Puck, Timbersaw, Earth Spirit, Lion]",False,Vici Gaming,"[Juggernaut, Void Spirit, Tidehunter, Shadow S...",True,Radiant
1,Map #2,6130109755,Vici Gaming,"[Terrorblade, Dragon Knight, Doom, Brewmaster,...",True,Team Aster,"[Phantom Lancer, Templar Assassin, Dark Seer, ...",False,Dire
2,Map #1,6130182512,Royal Never Give Up,"[Arc Warden, Viper, Timbersaw, Windranger, Bre...",True,PSG.LGD,"[Luna, Shadow Fiend, Tidehunter, Shadow Demon,...",False,Dire
3,Map #2,6130260292,PSG.LGD,"[Arc Warden, Bristleback, Pangolier, Keeper of...",False,Royal Never Give Up,"[Morphling, Invoker, Sand King, Hoodwink, Face...",True,Radiant
4,Map #1,6131631709,Royal Never Give Up,"[Arc Warden, Razor, Timbersaw, Windranger, Bre...",False,Invictus Gaming,"[Monkey King, Templar Assassin, Night Stalker,...",True,Radiant
...,...,...,...,...,...,...,...,...,...
4586,Map #2,7870204076,Nouns,"[Slark, Ember Spirit, Beastmaster, Rubick, Sna...",False,BOOM Esports,"[Gyrocopter, Dragon Knight, Brewmaster, Nyx As...",True,Radiant
4587,Map #3,7870252608,Nouns,"[Dragon Knight, Ember Spirit, Dark Seer, Jakir...",True,BOOM Esports,"[Templar Assassin, Mars, Weaver, Hoodwink, Clo...",False,Dire
4588,Map #1,7934129660,Gaimin Gladiators,"[Razor, Invoker, Broodmother, Snapfire, Io]",False,Nouns,"[Mirana, Storm Spirit, Doom, Hoodwink, Treant ...",True,Radiant
4589,Map #2,7934252472,Nouns,"[Weaver, Storm Spirit, Doom, Hoodwink, Phoenix]",False,Gaimin Gladiators,"[Windranger, Void Spirit, Broodmother, Sniper,...",True,Radiant


In [2]:
import json
import pandas as pd

with open('data/winrates.json') as file:
    winrates = json.load(file)


def get_feature_vec(winrates: dict, dire_pick: list, radiant_pick: list) -> list:
    dire_pick_synergy_features = get_synergy_features(winrates, dire_pick)
    radiant_pick_synergy_features = get_synergy_features(winrates, radiant_pick)
    dire_pick_duel_features, radiant_pick_duel_features = get_duel_features(
        winrates, dire_pick, radiant_pick
    )

    return (
        dire_pick_synergy_features
        + dire_pick_duel_features
        + radiant_pick_synergy_features
        + radiant_pick_duel_features
    )


def get_synergy_features(winrates: dict, pick: list) -> list:
    pick_copy = pick[::]
    synergy_features = []
    for h1 in pick:
        for h2 in pick_copy:
            synergy_features.append(winrates[h1][h2]["with_winrate"])
        del pick_copy[0]
    return synergy_features


def get_duel_features(winrates: dict, dire_pick: list, radiant_pick: list) -> tuple:
    dire_duel_features, radiant_duel_features = [], []
    for h1 in dire_pick:
        for h2 in radiant_pick:
            against_winrate = winrates[h1][h2].get("against_winrate", 0)
            dire_duel_features.append(against_winrate)
            radiant_duel_features.append(1 - against_winrate)
    return dire_duel_features, radiant_duel_features


In [4]:
get_feature_vec(winrates, df.iloc[0]['dire_heroes'], df.iloc[0]['radiant_heroes'])


[0.48,
 0.8,
 0.5,
 0.5,
 0.33,
 0.42,
 0.5,
 0.33,
 0.0,
 0.52,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.2,
 0.6,
 0.5,
 0.33,
 0.5,
 0.17,
 0.0,
 0.5,
 0.33,
 0.5,
 0.25,
 0.33,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.75,
 0.43,
 0.5,
 0.33,
 0.56,
 0.5,
 0.5,
 0.5,
 0.5,
 0.61,
 0.56,
 0.5,
 0.5,
 0.55,
 0.5,
 0.5,
 0.43,
 0.5,
 0.63,
 0.5,
 0.8,
 0.4,
 0.5,
 0.6699999999999999,
 0.5,
 0.83,
 1.0,
 0.5,
 0.6699999999999999,
 0.5,
 0.75,
 0.6699999999999999,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.25,
 0.5700000000000001,
 0.5,
 0.6699999999999999]

In [7]:
from sklearn.model_selection import train_test_split

X = df[['dire_heroes', 'radiant_heroes']]
y = df['dire_win']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((220, 2), (74, 2), (220,), (74,))

In [13]:
import numpy as np

with open('data/winrates.json') as file:
    winrates_dict = json.load(file)
    
    
def train_test_dataset(matches_dataset):
    filtered_dataset = matches_dataset[(matches_dataset['dire_heroes'].apply(len) > 0) & (matches_dataset['radiant_heroes'].apply(len) > 0)]
    
    train_df, test_df = train_test_split(filtered_dataset, test_size=0.2)
    
    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Test DataFrame shape: {test_df.shape}")
    
    train_df.to_pickle('data/train_df.pkl')
    test_df.to_pickle('data/test_df.pkl')
    
    return train_df, test_df

df = pd.read_pickle('data/matches.pkl')

train_df, test_df = train_test_dataset(matches_dataset=df)

Train DataFrame shape: (232, 9)
Test DataFrame shape: (59, 9)


In [15]:
import requests
import json

with open("data/heroes_decoder.json") as file:
    heroes_id_names = json.load(file)


def get_match_picks(match_id):
    response = requests.get(f'https://api.opendota.com/api/matches/{match_id}')
    if 'Internal Server Error' in response.text:
        raise ValueError("Open Dota crushed, refresh the page and try Test Yourself page")
    radiant_picks = [pick["hero_id"] for pick in response.json()['picks_bans'] if pick["is_pick"] and pick["team"] == 0]
    dire_picks = [pick["hero_id"] for pick in response.json()['picks_bans'] if pick["is_pick"] and pick["team"] == 1]

    radiant_picks_decode = []
    dire_picks_decode = []
    for id in radiant_picks:
        for key, value in heroes_id_names.items():
            if int(key) == int(id):
                radiant_picks_decode.append(value)
                break

    for id in dire_picks:
        for key, value in heroes_id_names.items():
            if int(key) == int(id):
                dire_picks_decode.append(value)
                break

    return {"dire": dire_picks_decode, 'radiant': radiant_picks_decode,
            "dire_team": response.json()['dire_team']['name'], 'radiant_team': response.json()['radiant_team']['name']}

In [16]:
get_match_picks("7935234871")

{'dire': ['Tusk', 'Invoker', 'Luna', 'Batrider', 'Beastmaster'],
 'radiant': ['Kunkka',
  'Treant Protector',
  'Mirana',
  'Earth Spirit',
  'Dark Willow'],
 'dire_team': '1w Team',
 'radiant_team': 'Team Zero'}

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

data_root = 'data'
train_data = TabularDataset(data_root + 'train_features.csv')
test_data = TabularDataset(data_root + 'test_features.csv')

predictor = TabularPredictor(label='class').fit(train_data=train_data)
predictions = predictor.predict(test_data)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score



def train_xgboost(train_df, test_df):
    # Разделяем признаки (X) и метки (y) в тренировочном наборе
    X_train = train_df.iloc[:, :-1]  # Все столбцы, кроме последнего (признаки)
    y_train = train_df.iloc[:, -1]   # Последний столбец (метки)
    
    # Разделяем признаки (X) и метки (y) в тестовом наборе
    X_test = test_df.iloc[:, :-1]    # Все столбцы, кроме последнего (признаки)
    y_test = test_df.iloc[:, -1]     # Последний столбец (метки)
    
    # Создание модели XGBoost для бинарной классификации
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Предсказания на тестовом наборе
    y_pred = model.predict(X_test)
    
    # Оценка модели по точности
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    return model


In [1]:
import json

with open('data/heroes_decoder.json', 'r') as file:
    data = json.load(file)

reversed_data = {v: k for k, v in data.items()}

with open('data/heroes_encoder.json', 'w') as file:
    json.dump(reversed_data, file, indent=4)

print("Ключи и значения успешно поменяны местами.")


Ключи и значения успешно поменяны местами.


In [2]:
with open('data/heroes_encoder.json') as file:
    id_heroes_names = json.load(file)

In [3]:
id_heroes_names['Slark']

'93'

In [4]:
import numpy as np


total_heroes = 125

picked_heroes = [1, 5, 23, 50, 120]

one_hot_encoded = np.zeros(total_heroes, dtype=int)

for hero in picked_heroes:
    one_hot_encoded[hero - 1] = 1

print(one_hot_encoded)

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0.]
